## 3.4 Block Wallet 사이트 만들기

### 1_Block Wallet Back-end 구축하기

In [1]:
from flask import Flask
from datetime import datetime
from flask import render_template
from flask import request
from flask import url_for
from flask import redirect

import requests
import json
import os
import pandas as pd


app = Flask(__name__, template_folder=os.getcwd())

@app.route('/', methods=['GET', 'POST'])
def login():

    if request.method=='POST':
        print("login 버튼을 누름")
        input_value = request.form.to_dict(flat=False)
        print("login 지갑주소 : " , input_value)
        
        ### 기존 user 정보 확인
        headers = {'Content-Type' : 'application/json; charset=utf-8'}
        # 수정 후
        res = requests.get("http://localhost:5000/chain", headers=headers)
        print("Chain Response:", res.json())  # 체인 상태 출력
#         print(res.text )
        print("*"*8)
        status_json = json.loads(res.text)
        status_json['chain']    
        tx_amount_l = []
        tx_sender_l = []
        tx_reciv_l  = []
        tx_time_l   = []

        for chain_index in range(len(status_json['chain'])):
            chain_tx = status_json['chain'][chain_index]['transactions']
            for each_tx in range(len(chain_tx)):
                tx_amount_l.append(chain_tx[each_tx]['amount'])
                tx_sender_l.append(chain_tx[each_tx]['sender'])
                tx_reciv_l.append(chain_tx[each_tx]['recipient'])
                tx_time_l.append(chain_tx[each_tx]['timestamp'])

        df_tx = pd.DataFrame()
        df_tx['timestamp'] = tx_time_l  
        df_tx['sender'] = tx_sender_l 
        df_tx['recipient'] = tx_reciv_l
        df_tx['amount'] = tx_amount_l   
        df_tx



        df_sended = pd.DataFrame(df_tx.groupby('sender')['amount'].sum()).reset_index()
        df_sended.columns = ['user','sended_amount']
        df_received= pd.DataFrame(df_tx.groupby('recipient')['amount'].sum()).reset_index()
        df_received.columns = ['user','received_amount']
        df_received

        df_status = pd.merge(df_received,df_sended, on ='user', how=  'outer').fillna(0)
        df_status['balance'] = df_status['received_amount']  - df_status['sended_amount']  
        df_status       
    
    
        if (df_status['user']==input_value['wallet_id'][0] ).sum() == 1:
            print("로그인성공")
            return render_template("wallet.html",  wallet_id = input_value['wallet_id'][0], 
                                                    wallet_value = df_status[df_status['user']== df_status['user'].iloc[0]]['balance'].iloc[0])
        else:
            return "잘못된 지갑주소입니다."
        
    return render_template('login.html')

@app.route('/wallet', methods=['GET', 'POST'])
def wallet():
    if request.method=='POST':
        send_value = int(request.form.to_dict(flat=False)['send_value'][0] )
        send_target = request.form.to_dict(flat=False)['send_target'][0]
        send_from = request.form.to_dict(flat=False)['send_from'][0]
        print("123123 : " ,send_from)
        
        if send_value > 0:
            print(send_value)
            ## transaction 입력하기
            headers = {'Content-Type' : 'application/json; charset=utf-8'}
            data = {
                "sender": send_from,
                "recipient": send_target,
                "amount": send_value,
            }
            requests.post("http://localhost:5000/transactions/new", headers=headers, data=json.dumps(data)).content

            return "전송 완료!"

        else:
            return "0 pyBTC 이상 보내주세요!"

        
        
    return render_template('wallet.html')
    
    
    
app.run(port=8081)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8081
Press CTRL+C to quit
127.0.0.1 - - [23/Sep/2024 20:14:29] "GET / HTTP/1.1" 200 -


login 버튼을 누름
login 지갑주소 :  {'wallet_id': ['test-to']}


127.0.0.1 - - [23/Sep/2024 20:14:34] "POST / HTTP/1.1" 200 -


Chain Response: {'chain': [{'index': 1, 'nonce': 100, 'previous_hash': 1, 'timestamp': 1727089816.295097, 'transactions': []}, {'index': 2, 'nonce': 9317841, 'previous_hash': 'e85f9a67c9d69f06eafa0089baf6f5d52dc1ffe4d1b07ccd72706ce7276cb921', 'timestamp': 1727089826.2203434, 'transactions': [{'amount': 3, 'recipient': 'test-to', 'sender': 'test-from', 'timestamp': 1727089822.063624}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089826.2203434}]}, {'index': 3, 'nonce': 770304, 'previous_hash': '8c28c7739040cb0f2e1bd97e95e559327db2b2156bafd83b6038992497761920', 'timestamp': 1727089834.5278556, 'transactions': [{'amount': 30, 'recipient': 'test-to2', 'sender': 'test-from', 'timestamp': 1727089830.341018}, {'amount': 300, 'recipient': 'test-to3', 'sender': 'test-from', 'timestamp': 1727089832.3899186}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089834.5278556}]}], 'length': 3}
********
로그인성공
123123 :  test-to
3


...

Chain Response: {'chain': [{'index': 1, 'nonce': 100, 'previous_hash': 1, 'timestamp': 1727089816.295097, 'transactions': []}, {'index': 2, 'nonce': 9317841, 'previous_hash': 'e85f9a67c9d69f06eafa0089baf6f5d52dc1ffe4d1b07ccd72706ce7276cb921', 'timestamp': 1727089826.2203434, 'transactions': [{'amount': 3, 'recipient': 'test-to', 'sender': 'test-from', 'timestamp': 1727089822.063624}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089826.2203434}]}, {'index': 3, 'nonce': 770304, 'previous_hash': '8c28c7739040cb0f2e1bd97e95e559327db2b2156bafd83b6038992497761920', 'timestamp': 1727089834.5278556, 'transactions': [{'amount': 30, 'recipient': 'test-to2', 'sender': 'test-from', 'timestamp': 1727089830.341018}, {'amount': 300, 'recipient': 'test-to3', 'sender': 'test-from', 'timestamp': 1727089832.3899186}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089834.5278556}]}], 'length': 3}
********
로그인성공


127.0.0.1 - - [23/Sep/2024 20:35:56] "GET / HTTP/1.1" 200 -


login 버튼을 누름
login 지갑주소 :  {'wallet_id': ['test-to']}


127.0.0.1 - - [23/Sep/2024 20:36:01] "POST / HTTP/1.1" 200 -


Chain Response: {'chain': [{'index': 1, 'nonce': 100, 'previous_hash': 1, 'timestamp': 1727089816.295097, 'transactions': []}, {'index': 2, 'nonce': 9317841, 'previous_hash': 'e85f9a67c9d69f06eafa0089baf6f5d52dc1ffe4d1b07ccd72706ce7276cb921', 'timestamp': 1727089826.2203434, 'transactions': [{'amount': 3, 'recipient': 'test-to', 'sender': 'test-from', 'timestamp': 1727089822.063624}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089826.2203434}]}, {'index': 3, 'nonce': 770304, 'previous_hash': '8c28c7739040cb0f2e1bd97e95e559327db2b2156bafd83b6038992497761920', 'timestamp': 1727089834.5278556, 'transactions': [{'amount': 30, 'recipient': 'test-to2', 'sender': 'test-from', 'timestamp': 1727089830.341018}, {'amount': 300, 'recipient': 'test-to3', 'sender': 'test-from', 'timestamp': 1727089832.3899186}, {'amount': 0.1, 'recipient': 'node_5000', 'sender': 'master', 'timestamp': 1727089834.5278556}]}, {'index': 4, 'nonce': 8417091, 'previous_hash': 'c351f8ae3